# Learn v75 med walkthrough-metoden

In [1]:
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier,Pool,cv,utils
import pickle
import sys
sys.path.append('C:\\Users\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel')
import V75_scraping as vs


END


In [2]:

def proba_order_score(df_, y, proba):  # df skall innehålla datum,avd,vodds
    kassa=1000
    df = df_.copy()
    df['proba'] = proba[:,1]
    df['f'] = (df.proba*df.vodds - 1) / (df.vodds-1)  # kelly formel
    df['spela'] = df.f >0
    df['insats'] = df.spela * df.f * kassa

    df.sort_values(['datum','avd','proba'],ascending=[True,True,False],inplace=True)
    proba_order=df.groupby(['datum','avd']).proba.cumcount()

    df['prob_order']=proba_order+1
    df['y'] = y

    print('log(proba)',np.log(df.loc[df.y==1].proba).mean())
    return df, df.loc[df.y==1].prob_order.mean()   # mean prob_order för vinnarhäst


In [3]:
def create_ekipage(df_):
    df=df_.copy()
    prefs = ['','h1_','h2_','h3_','h4_','h5_',]
    for pr in prefs:
        df[pr+'ekipage'] = df[pr+'kusk'].str.cat(df['häst'], sep =", ")
        df.drop([pr+'kusk'],axis=1, inplace=True)
        
    return df.drop(['häst'], axis=1)

In [4]:
### returnera en modell med parametrar satta
def get_model(d=6,l2=2,iterations=3000,use_best=True,verbose=False):
    model = CatBoostClassifier(iterations=iterations,use_best_model=use_best, 
        custom_metric=['Logloss', 'AUC','Recall', 'Precision', 'F1', 'Accuracy'],

        eval_metric='Accuracy', 
        depth=d,l2_leaf_reg=l2,
        auto_class_weights='Balanced',verbose=verbose, random_state=2021) 
    return model                

In [5]:
### Features som inte används vid träning
def remove_features(df_,remove_mer=[]):
    # df = df_.copy()
    #remove_mer=['h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf']
    df = df_.drop(['avd','startnr','vodds','podds','bins','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1) #
    if remove_mer:
        df = df.drop(remove_mer,axis=1)
    
    # df=check_unique(df.copy())
    # df=check_corr(df.copy())
    return df

In [6]:
 ## byt ut alla NaN till text för cat_features
def replace_NaN(X_train,X_test=None, cat_features=[]):
    # print('cat_features',cat_features)
    for c in cat_features:
        # print(c)
        X_train.loc[X_train[c].isna(),c] = 'None'       ### byt ut None-värden till texten 'None
        if X_test is not None:  ## om X_test är med
            X_test.loc [X_test[c].isna(),c] = 'None'    ### byt ut None-värden till texten 'None

    return X_train,X_test

In [7]:
def scrape_nya_lopp(features=None, target=None,save=True):
   """scrape nya lopp och lägg in i all_data.csv
      concat matchar ihop samma kolumner oavsett ordning!
   """
   nya_lopp,strukna = vs.v75_scraping(resultat=True,history=True)
   
   print('nya',nya_lopp.shape)
      
   dfall = pd.read_csv('all_data.csv')
   assert dfall.shape[1] == nya_lopp.shape[1], f'antal kolumner {dfall.shape[1]} resp {nya_lopp.shape[1]} skall vara lika'
   assert set(dfall.columns) == set(nya_lopp.columns), ' det skall vara samma kolumnnamn'
   nya_lopp = nya_lopp[dfall.columns]
   dfconc = pd.concat([dfall, nya_lopp])
   assert dfconc.shape[0] == dfall.shape[0] + nya_lopp.shape[0], 'antal rader skall vara summan'
   assert dfconc.shape[1] == dfall.shape[1], 'antal kolumner skall vara oförändrat'
   
   #ta bort dubletter
   dfconc.drop_duplicates(['datum','avd','häst'],inplace=True)
   dfconc.sort_values(by=['datum','avd'],inplace=True)
   dfconc.reset_index(drop=True,inplace=True)
   print('shape efter dubletter bort',dfconc.shape)

   if save:
      dfconc.to_csv('all_data.csv', index=False)

   print("första datum i df =",dfconc.datum.head(1).to_list()[0])
   print("sista  datum i df =",dfconc.datum.tail(1).to_list()[0])

   return dfconc,nya_lopp,strukna


In [8]:
### beräkna vilka datum att använda ###
def get_data(test_from_proc=0.75, train_from_proc=0, total_omlärning = False):
    if total_omlärning:
        nya_lopp=None
        df = pd.read_csv('all_data.csv')     
        datum_att_lära = df.datum.unique()
        split_ix = int(len(datum_att_lära)*test_from_proc)
    else:
        # normalt adderar vi bara 1 eller flera veckor från "omg_att_spela_link.csv"
        df, nya_lopp, strukna = scrape_nya_lopp()  # scrape från 'omg_att_spela_link.csv' och addera till df
        omg_df = pd.read_csv('omg_att_spela_link.csv')     
        startix=omg_df.Link.str.find('spel')[0]    # index till 'spel' i url
        datum_att_lära = omg_df.Link.str.slice(start=startix+5,stop=startix+15).to_list() # en datum 
        split_ix=0
        print(f'datum att lära: {datum_att_lära}')

    return df,nya_lopp,datum_att_lära,split_ix


## Walkthrough-funktionen  här

In [ ]:

### Kör en walkthrough learn här, en datum i taget framåt

# Jag har ändrat till att alla steg kör utan test-datam ed fast iterations=100
def walkthrough(classic_test=False, verbose=False):
    
    df, nya_lopp, alla_datum, split_ix = get_data(0.8)
    # Läs in en fil
    

    l2_leaf_regs=2
    model=get_model(use_best=False,iterations=100)
    df=remove_features(df.copy())
    cat_features = list(df.loc[:,df.dtypes=='O'].columns)
    df,_ = replace_NaN(df.copy(), cat_features=cat_features)    
    print(f'cat_features {cat_features}\n')

    df['plac']=(df.plac==1)*1
        
    for nr,datum in enumerate(alla_datum[split_ix:]):
        print(f'walk-iter {nr+1} av {len(alla_datum[split_ix:])} ',end=': ')

        X_train = df.loc[df.datum<datum,:].copy()
        y_train = X_train.plac; X_train.drop(['plac'],axis=1,inplace=True)

        if classic_test:    ### klassisk train/test utan walkthrough
            X_test  = df.loc[df.datum>=datum,:].copy()
            y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
            train_pool = Pool(X_train,y_train,cat_features=cat_features)
            test_pool = Pool(X_test,y_test,cat_features=cat_features)
            model.fit(train_pool,use_best_model=True, verbose=verbose,eval_set=test_pool)
        else:
            X_test  = df.loc[df.datum==datum,:].copy()
            y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
            train_pool = Pool(X_train,y_train,cat_features=cat_features)
            test_pool = Pool(X_test,y_test,cat_features=cat_features)
            model.fit(train_pool,use_best_model=False, verbose=verbose)

        print('best iteration',model.get_best_iteration(), '\tbest score', round(model.get_best_score()['learn']['Accuracy'],3) )
        ##['validation']['Logloss'],3),'\t', round(model.get_best_score()['validation']['Accuracy:use_weights=true'],3))
        
        if classic_test:    ### klassisk train/test utan walkthrough
            return model,cat_features
    
        model.save_model('modeller/model_'+datum)

    X_train =df.copy().drop('plac',axis=1)
    y_train = df.plac 
    model.fit(X_train,y=y_train,cat_features=cat_features)
    print(f'spara model_senaste',datum)
    model.save_model('modeller/model_senaste')

    return df,nya_lopp, model,cat_features

### Här körs hela walkthrough

In [ ]:
df, nya_lopp, model, cat_features = walkthrough(classic_test=False, verbose=False)

## Kör allt ovanför walkthrough
### Se till att "omg_att_spela_link.csv" är ifylld

## init  - kör först allt t.o.m 'replace_NaN()' ovan

In [ ]:
model = get_model().load_model('modeller/model_senaste')
dforg = pd.read_csv('all_data.csv')     
# print(df.columns)
df=remove_features(dforg.copy())
# df['avd']=dforg.avd
cat_features = list(df.loc[:,df.dtypes=='O'].columns)
df,_ = replace_NaN(df.copy(), cat_features=cat_features)    
y=df.plac
y=(y==1)*1
df.drop('plac',axis=1,inplace=True)

## cv

In [ ]:

cv_pool = Pool(df,y,cat_features=cat_features)

params = {
         'use_best_model': True,
         'eval_metric' : 'AUC',
         "loss_function": "Logloss",
         'early_stopping_rounds': 100,
         'verbose': 50,
}

cv_score =cv(pool=cv_pool, 
   params=params, 
   dtrain=None, 
   iterations=2000, 
   num_boost_round=None,
   fold_count=5, 
   nfold=None,
   inverted=False,
   partition_random_seed=0,
   seed=2021, 
   shuffle=False, 
   logging_level=None, 
   stratified=True,
   as_pandas=True,
   type='TimeSeries')

In [ ]:
cv_score

In [ ]:
from IPython.display import display
print(df.datum.min(),df.datum.max())
display(cv_score[cv_score['test-Logloss-mean'].min() == cv_score['test-Logloss-mean']])
display(cv_score[cv_score['test-AUC-mean'].max() == cv_score['test-AUC-mean']])

In [ ]:
from sklearn.model_selection import train_test_split

df[['datum','avd','streck','häst','kusk']] = dforg[['datum','avd','streck','häst','kusk']]

# df.drop('datum',axis=1,inplace=True)
df.drop('avd',axis=1,inplace=True)
df.drop(['streck'],axis=1,inplace=True)
# df.drop(['häst','kusk'],axis=1,inplace=True)
cat_features = list(df.loc[:,df.dtypes=='O'].columns)
X_train,X_test,y_train,y_test = train_test_split(df,y,shuffle=False,)
print("test:",X_test.datum.min(),X_test.datum.max())
print("train:",X_train.datum.min(),X_train.datum.max())
cb=get_model(use_best=True)
cb.fit(X_train,y_train,eval_set= (X_test,y_test),early_stopping_rounds=200, cat_features=cat_features,verbose=100)

In [ ]:
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
predict_prob = cb.predict_proba(X_test)

_,prob_score = proba_order_score(X_test ,y_test, predict_prob)

print('cb med ekipage',prob_score, cb.best_score_['validation']['AUC'])

# FLAML (med och utan ekipage och streck)
För att köra enbart FLAML initiera först med allt innan plus walkthrough  

In [ ]:
def ordinal_enc(df_, features):
    df = df_.copy()
    from sklearn.preprocessing import OrdinalEncoder
    enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=100000)
    enc.fit(df[[features]])
    df[features] = enc.transform(df[[features]])
    return df,enc

# df,enc = ordinal_enc(dforg,'häst')
# import pickle
# pickle.dump(enc, open('modeller/encoder.sav', 'wb'))


In [ ]:
def split_data(df_,train_from_proc=0,test_proc=0.25):
    # train_from_proc = where to start both train and test
    # test_proc = how much of the data is test
    df=df_.copy()
    alla_datum = df.datum.unique()
    train_from_datum = alla_datum[ int(len(alla_datum)*train_from_proc)]
    X_test=None
    y_test=None
    
    if test_proc:
        selected_data = alla_datum[ alla_datum >= train_from_datum ]
        test_from_datum = selected_data[ int(len(selected_data)*(1-test_proc)) ]
        X_test  = df[df.datum >= test_from_datum]
        y_test  = (X_test.plac==1)*1
        X_test  = X_test.drop('plac',axis=1)
        print(f'test from {X_test.datum.min()} to {X_test.datum.max()} (incl)')
    
        X_train = df[(df.datum >= train_from_datum) & (df.datum < test_from_datum) ]
    else:
        print('Only train data - No test')
        X_train = df[(df.datum >= train_from_datum)]
    
    y_train = (X_train.plac==1)*1
    
    print(f'train from {X_train.datum.min()} to {X_train.datum.max()} (incl)')
    
    return X_train.drop('plac', axis=1), X_test, y_train, y_test


In [ ]:
# prepare all data för flaml
dforg = pd.read_csv('all_data.csv')  

### enc is the encoder that we will save for use during v75_spel.py ###
### It will be used and finally saved later in this code ###
df,env = ordinal_enc(dforg,'häst')

X_train, X_test, y_train, y_test= split_data(df,train_from_proc=0,test_proc=0.25)
X_train = remove_features(X_train)
X_test  = remove_features( X_test)

# X_train = X_train.drop('streck', axis=1)
# X_test  = X_test.drop ('streck', axis=1)
# X_train.drop('datum', axis=1, inplace=True)
# X_test.drop( 'datum', axis=1, inplace=True)
cat_features = list(X_train.select_dtypes('object').columns)
# X_train, X_test = replace_NaN(X_train.copy(),X_test=X_test.copy(), cat_features=cat_features) 
# X_train.fillna(-1)
# X_test.fillna(-1)
print(cat_features)
X_train.shape,X_test.shape

In [ ]:
from flaml import AutoML 
cat_features = list(X_train.select_dtypes('object').columns)
starting_points={'lgbm': {'n_estimators': 38,
  'num_leaves': 4,
  'min_child_samples': 2,
  'learning_rate': 0.19098448074739216,
  'log_max_bin': 7,
  'colsample_bytree': 0.8827412174089042,
  'reg_alpha': 0.004577823970660193,
  'reg_lambda': 0.03815584533462228},
 'rf': {'n_estimators': 33,
  'max_features': 0.3251674877768946,
  'max_leaves': 89,
  'criterion': 'entropy'},
 'catboost': {'early_stopping_rounds': 50,
  'learning_rate': 0.007511731949060241},
 'xgboost': {'n_estimators': 575,
  'max_leaves': 46,
  'min_child_weight': 1.032235057697502,
  'learning_rate': 0.013318439439138472,
  'subsample': 0.7908401179782586,
  'colsample_bylevel': 0.6924750037579576,
  'colsample_bytree': 0.7174828796230647,
  'reg_alpha': 0.15461500385937774,
  'reg_lambda': 0.6619886587472544},
 'extra_tree': {'n_estimators': 47,
  'max_features': 0.7934349565988307,
  'max_leaves': 213,
  'criterion': 'entropy'}}
flml_raw_parms={'task': 'classification','split_type':'time', 'metric':'roc_auc', 'starting_points': starting_points,'verbose':False,
        'time_budget':1200, 'max_iter':50000000,'n_jobs':5, 'X_val': X_test, 'y_val':y_test,'early_stop':True, 'ensemble':True}

automl_raw = AutoML()
automl_raw.fit(X_train,y_train, **flml_raw_parms)

In [ ]:
flm_raw_train_pred= automl_raw.predict_proba(X_train)
flm_raw_test_pred = automl_raw.predict_proba(X_test)

X_test_raw = X_test.copy()
X_test_raw[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
_,prob_score = proba_order_score(X_test_raw,y_test, flm_raw_test_pred)

print('timeserie, datum,häst, kusk', prob_score, 1-automl_raw.best_loss)
# X_test.columns

## Final FLML models

In [ ]:
def save_best_config(best_config):
    # save best_config
    import pickle
    with open('best_config_per_estimator.sav', "wb") as f:
        pickle.dump(best_config, f)

In [ ]:
def run_flaml(X_train, y_train, df_perf, save=True):
    # read best_config
    with open('best_config_per_estimator.sav', "rb") as f:
        best_config = pickle.load(f)
        
    from_date = X_train.datum.min()
    to_date = X_train.datum.max()
    automl = [None,None]
    for with_streck in [True, False]:
        if with_streck: 
            X_tr = X_train.copy()    
            filename = 'modeller\\FLAML_model.sav'
        else:
            X_tr = X_train.drop('streck', axis=1).copy()
            filename = 'modeller\\FLAML2_model.sav'
            
        print('with_streck = ',with_streck)   
    
        automl[with_streck] = AutoML()
        
        with open('best_config_per_estimator.sav', "rb") as f:
            best_config = pickle.load(f)
            
        flml_parms={'task': 'classification','split_type':'time', 'metric':'roc_auc','starting_points': best_config[with_streck], 'verbose':False,
        'time_budget':2600,'n_jobs':5, 'early_stop':True, 'ensemble':True}

        automl[with_streck].fit(X_tr, y_train, **flml_parms)
        perf = 1-automl[with_streck].best_loss
        print(perf, 'for streck in columns', with_streck)
        df_perf.loc[len(df_perf)] = [from_date, to_date, with_streck, perf]
        
        # save_model
        if save:
            print('save model in',filename)   
            with open(filename,"wb") as f:
                pickle.dump(automl[with_streck], f, pickle.HIGHEST_PROTOCOL)
                
            print('save encoder enc in encoder.sav')
            with open('encoder.sav',"wb") as f:
                pickle.dump(enc, f, pickle.HIGHEST_PROTOCOL)
             
    save_best_config(best_config)  

    # remove duplicates
    df_perf.drop_duplicates(subset=['learn_from','learn_to','streck'], keep='last', inplace=True)

    print('\n','\n')
    display(df_perf.tail(30).sort_values(by=['perf'], ascending=False))
    print('Med streck max:',df_perf.loc[df_perf.streck == True].perf.max())
    print('Ej  streck max:',df_perf.loc[df_perf.streck == False].perf.max())

    df_perf.to_csv('perf_flaml.csv', index=False)
    return automl

In [ ]:
# prepare all data för flaml
dforg = pd.read_csv('all_data.csv')  
df,enc = ordinal_enc(dforg, 'häst')
X_train, _, y_train, _ = split_data(df,train_from_proc=0,test_proc=None)
X_train = remove_features(X_train)

df_perf = pd.read_csv('perf_flaml.csv')
automl = run_flaml(X_train, y_train, df_perf)


# remove garbage

In [ ]:
def remove_garbage():
    import subprocess
    subprocess.call([r'C:/Users/peter/Documents/MyProjects/PyProj/Trav/spel/remove_dirt.bat'])
remove_garbage()    

# Tester

In [9]:
### kollar att scraping av flera datum samtidigt fungerar
df, nya_lopp, strukna = scrape_nya_lopp()  # scrape från 'omg_att_spela_link.csv' och addera till df

omgång 1: https://www.atg.se/spel/2022-03-19/V75/
klickade på Anpassa
hoppar över voods click (verkar vara förifyllt
distans och spår är enabled
distans och spår är displayed
DISTANS OCH SPÅR ej selected ännu
DISTANS OCH SPÅR är fel
efter click distans och spår
Prova name checkbox-ageAndSex
 är enabled
 är korrekt valt
 är korrekt
efter click Spara
ant resultat 7
ant lopp 7
EUR: False NOK: True
priser ['110000-55000-32000-19000-12500-10000-6500-5000 NOK', '110000-55000-32000-19000-12500-10000-6500-5000 NOK', '150000-75000-40000-25000-15000-11500-7500-5000 NOK', '110000-55000-32000-19000-12500-10000-6500-5000 NOK', '110000-55000-32000-19000-12500-10000-6500-5000 NOK', '200000-100000-55000-30000-20000-12500-8000-6000 NOK', '110000-55000-32000-19000-12500-10000-6500-5000 NOK']
Ant priser 7
pris: 110000
ant names,vodds,podds,rader,streck 15 15 15 15
AVD 1 MOMARKEN 3140 VOLTSTART ...............
pris: 110000
ant names,vodds,podds,rader,streck 12 12 12 12
AVD 2 MOMARKEN 2140 AUTOSTART ......

(44792, 79)

In [ ]:
# Kolla autmoml-grejer från dokumntationen
#### How much time is needed to find the best model​
#### If you want to get a sense of how much time is needed to find the best model, you can use max_iter = 2 to perform two trials first.
#### You will see the time to finish the first and cheapest trial in seconds. 
#### The estimated necessary time budget in, and the estimated sufficient time budget inseconds. 
def set_time_budget():
  import time
  start_time = time.time()

  automl = AutoML()

  with open('best_config_per_estimator.sav', "rb") as f:
      best_config = pickle.load(f)
      
  flml_parms = {'task': 'classification', 'split_type': 'time', 'metric': 'roc_auc', 'starting_points': best_config[0], 
                'time_budget':2600, 'n_jobs': 5, 'early_stop': True, 'ensemble': True}
              # 'time_budget': 1700, 'max_iter': 400000000, 'n_jobs': 5, 'early_stop': True, 'ensemble': True,'verbose': True,}

  automl.fit(X_train, y_train, log_file_name='flaml_log.json', **flml_parms)

  return automl, time.time() - start_time

tid=None
# automl, tid = set_time_budget()  # tid in sekunder
print('tid', tid)

In [ ]:
print('best_estimator:', automl.best_estimator)
print('best eest. config:',automl.best_config)
print('\nbest per est.:',automl.best_config_per_estimator)
print('\nbest_config train time:',automl.best_config_train_time)
print('best iteration:',automl.best_iteration)
print('best loss:',automl.best_loss)
print('time to find best mod.',automl.time_to_find_best_model)
print('\nhistory:',automl.config_history)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from flaml.data import get_output_from_log
time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = get_output_from_log(
    filename="flaml_log.json", time_budget=120)

plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Accuracy")
plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
plt.show()


## Hur skall vi köra den ny modellen 
1. preprocessa datat (nya kolumner)
    - proba och eller Kelly, ant hästar i loppet, favoriter, bara solklara favoriter
2. bestäm cat_features
3. Kör catboost eller flaml?
   - om flaml träna upp den
   - 

In [ ]:
remove_garbage()